In [1]:
import gc
import pandas as pd
import numpy as np
from datetime import datetime

from xgboost import XGBClassifier
from sklift.models import ClassTransformation

from sklift.metrics import uplift_at_k
import matplotlib.pyplot as plt
import catboost as cb
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import optuna
import pandas as pd
from pandas import read_csv

%matplotlib inline

In [2]:
data = read_csv('dataset.csv')
y = data['response_att']
X = data.copy()


features_list = ['group', 'response_att',
 'perdelta_days_between_visits_15_30d',
 'k_var_days_between_visits_1m',
 'k_var_days_between_visits_3m',
 'k_var_days_between_visits_15d',
 'cheque_count_6m_g48',
 'response_sms',
 'cheque_count_6m_g40',
 'k_var_count_per_cheque_6m_g27',
 'k_var_disc_share_6m_g27',
 'response_viber',
 'k_var_discount_depth_1m',
 'k_var_sku_price_6m_g48',
 'cheque_count_6m_g25',
 'cheque_count_12m_g41',
 'cheque_count_6m_g41',
 'cheque_count_12m_g32',
 'sale_count_12m_g54',
 'cheque_count_6m_g32',
 'k_var_sku_price_6m_g26',
 'k_var_disc_share_6m_g24',
 'food_share_1m',
 'k_var_cheque_3m',
 'age',
 'stdev_days_between_visits_15d',
 'cheque_count_6m_g38',
 'cheque_count_3m_g57',
 'cheque_count_12m_g25',
 'months_from_register',
 'main_format',
 'food_share_15d',
 'cheque_count_12m_g48',
 'k_var_disc_share_15d_g24',
 'sale_count_12m_g49',
 'cheque_count_6m_g21',
 'crazy_purchases_goods_count_12m',
 'cheque_count_3m_g25',
 'sale_count_3m_g57',
 'k_var_sku_price_3m_g49',
 'k_var_sku_price_15d_g49',
 'k_var_disc_share_1m_g40',
 'k_var_disc_share_3m_g24',
 'k_var_sku_price_6m_g49',
 'cheque_count_6m_g42'
 'k_var_cheque_15d'
]

additional_list = ['response_att',
 'k_var_days_between_visits_15d',
 'k_var_days_between_visits_1m',
 'k_var_disc_per_cheque_15d',
 'cheque_count_12m_g41',
 'cheque_count_6m_g41',
 'cheque_count_6m_g32',
 'cheque_count_6m_g48',
 'cheque_count_6m_g40',
 'cheque_count_12m_g46',
 'k_var_cheque_15d',
 'cheque_count_6m_g46',
 'cheque_count_12m_g48',
 'cheque_count_6m_g33',
 'cheque_count_12m_g32',
 'cheque_count_12m_g33',
 'cheque_count_12m_g38',
 'cheque_count_6m_g38',
 'stdev_discount_depth_15d',
 'k_var_sku_per_cheque_15d']

features_list = list(set(additional_list).union(set(features_list)))

#X = X.filter(features_list)
pd.set_option('display.max_columns', 500)
X.head(5)

,Unnamed: 0,CardHolder,age,cheque_count_12m_g20,cheque_count_12m_g21,cheque_count_12m_g25,cheque_count_12m_g32,cheque_count_12m_g33,cheque_count_12m_g38,cheque_count_12m_g39,cheque_count_12m_g41,cheque_count_12m_g42,cheque_count_12m_g45,cheque_count_12m_g46,cheque_count_12m_g48,cheque_count_12m_g52,cheque_count_12m_g56,cheque_count_12m_g57,cheque_count_12m_g58,cheque_count_12m_g79,cheque_count_3m_g20,cheque_count_3m_g21,cheque_count_3m_g25,cheque_count_3m_g42,cheque_count_3m_g45,cheque_count_3m_g52,cheque_count_3m_g56,cheque_count_3m_g57,cheque_count_3m_g79,cheque_count_6m_g20,cheque_count_6m_g21,cheque_count_6m_g25,cheque_count_6m_g32,cheque_count_6m_g33,cheque_count_6m_g38,cheque_count_6m_g39,cheque_count_6m_g40,cheque_count_6m_g41,cheque_count_6m_g42,cheque_count_6m_g45,cheque_count_6m_g46,cheque_count_6m_g48,cheque_count_6m_g52,cheque_count_6m_g56,cheque_count_6m_g57,cheque_count_6m_g58,cheque_count_6m_g79,children,crazy_purchases_cheque_count_12m,crazy_purchases_cheque_count_1m,crazy_purchases_cheque_count_3m,crazy_purchases_cheque_count_6m,crazy_purchases_goods_count_12m,crazy_purchases_goods_count_6m,disc_sum_6m_g34,food_share_15d,food_share_1m,gender,group,k_var_cheque_15d,k_var_cheque_3m,k_var_cheque_category_width_15d,k_var_cheque_group_width_15d,k_var_count_per_cheque_15d_g24,k_var_count_per_cheque_15d_g34,k_var_count_per_cheque_1m_g24,k_var_count_per_cheque_1m_g27,k_var_count_per_cheque_1m_g34,k_var_count_per_cheque_1m_g44,k_var_count_per_cheque_1m_g49,k_var_count_per_cheque_3m_g24,k_var_count_per_cheque_3m_g27,k_var_count_per_cheque_3m_g32,k_var_count_per_cheque_3m_g34,k_var_count_per_cheque_3m_g41,k_var_count_per_cheque_3m_g44,k_var_count_per_cheque_6m_g24,k_var_count_per_cheque_6m_g27,k_var_count_per_cheque_6m_g32,k_var_count_per_cheque_6m_g44,k_var_days_between_visits_15d,k_var_days_between_visits_1m,k_var_days_between_visits_3m,k_var_disc_per_cheque_15d,k_var_disc_share_12m_g32,k_var_disc_share_15d_g24,k_var_disc_share_15d_g34,k_var_disc_share_15d_g49,k_var_disc_share_1m_g24,k_var_disc_share_1m_g27,k_var_disc_share_1m_g34,k_var_disc_share_1m_g40,k_var_disc_share_1m_g44,k_var_disc_share_1m_g49,k_var_disc_share_1m_g54,k_var_disc_share_3m_g24,k_var_disc_share_3m_g26,k_var_disc_share_3m_g27,k_var_disc_share_3m_g32,k_var_disc_share_3m_g33,k_var_disc_share_3m_g34,k_var_disc_share_3m_g38,k_var_disc_share_3m_g40,k_var_disc_share_3m_g41,k_var_disc_share_3m_g44,k_var_disc_share_3m_g46,k_var_disc_share_3m_g48,k_var_disc_share_3m_g49,k_var_disc_share_3m_g54,k_var_disc_share_6m_g24,k_var_disc_share_6m_g27,k_var_disc_share_6m_g32,k_var_disc_share_6m_g34,k_var_disc_share_6m_g44,k_var_disc_share_6m_g46,k_var_disc_share_6m_g49,k_var_disc_share_6m_g54,k_var_discount_depth_15d,k_var_discount_depth_1m,k_var_sku_per_cheque_15d,k_var_sku_price_12m_g32,k_var_sku_price_15d_g34,k_var_sku_price_15d_g49,k_var_sku_price_1m_g24,k_var_sku_price_1m_g26,k_var_sku_price_1m_g27,k_var_sku_price_1m_g34,k_var_sku_price_1m_g40,k_var_sku_price_1m_g44,k_var_sku_price_1m_g49,k_var_sku_price_1m_g54,k_var_sku_price_3m_g24,k_var_sku_price_3m_g26,k_var_sku_price_3m_g27,k_var_sku_price_3m_g32,k_var_sku_price_3m_g33,k_var_sku_price_3m_g34,k_var_sku_price_3m_g40,k_var_sku_price_3m_g41,k_var_sku_price_3m_g44,k_var_sku_price_3m_g46,k_var_sku_price_3m_g48,k_var_sku_price_3m_g49,k_var_sku_price_3m_g54,k_var_sku_price_6m_g24,k_var_sku_price_6m_g26,k_var_sku_price_6m_g27,k_var_sku_price_6m_g32,k_var_sku_price_6m_g41,k_var_sku_price_6m_g42,k_var_sku_price_6m_g44,k_var_sku_price_6m_g48,k_var_sku_price_6m_g49,main_format,mean_discount_depth_15d,months_from_register,perdelta_days_between_visits_15_30d,promo_share_15d,response_att,response_sms,response_viber,sale_count_12m_g32,sale_count_12m_g33,sale_count_12m_g49,sale_count_12m_g54,sale_count_12m_g57,sale_count_3m_g24,sale_count_3m_g33,sale_count_3m_g57,sale_count_6m_g24,sale_count_6m_g25,sale_count_6m_g32,sale_count_6m_g33,sale_count_6m_g44,sale_count_6m_g54,sale_count_6m_g57,sale_sum_12m_g24,sale_sum_12m_g25,sale_s

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [4]:
import azureml.core
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Experiment, Workspace

# Check core SDK version number
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")
print("")

# Log In to Azure ML Workspace
interactive_auth = InteractiveLoginAuthentication(tenant_id="76f90eb1-fb9a-4446-9875-4d323d6455ad")

ws = Workspace.from_config(auth=interactive_auth)
print('Workspace name: ' + ws.name, sep='\n')

You are currently using version 1.5.0 of the Azure ML SDK

Workspace name: team19


In [5]:
def custom_metric(answers, take_top_ratio=0.25):
    answers.sort_values(by='uplift', inplace=True, ascending=False)
    
    n_samples = int(np.ceil(answers.shape[0] * take_top_ratio))
    answers = answers.iloc[:n_samples, :]
    answers_test = answers[answers['group'] == 1]['response_att'].sum() / \
                   answers[answers['group'] == 1].shape[0]
    answers_control = answers[answers['group'] == 0]['response_att'].sum() / \
                      answers[answers['group'] == 0].shape[0]
    return (answers_test - answers_control) * 100

In [6]:
# Get an experiment object from Azure Machine Learning

In [7]:
#i = 0

In [9]:
import numpy as np
from tqdm import tqdm
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


# list of numbers from 0 to 1.0 with a 0.05 interval
lrs = [0.17]
depth = [4]
num_estimators = list(np.arange(190, 215, 5))

# try a bunch of alpha values in a Linear Regression (Ridge) model
experiment = Experiment(workspace=ws, name="cycling_features_all")
max_score = 0
for max_depth in depth:
    for n_estimators in num_estimators:
        for lr in lrs:
            print('lr', lr)
            print('depth', max_depth)

            with experiment.start_logging(snapshot_directory=None) as run:
                run.log(name='lr', value=lr)
                run.log(name='depth', value=max_depth)
                run.log(name='n_estimators', value=n_estimators)

                xgb_est_params = {
                    'max_depth':max_depth,
                    'learning_rate': lr, 
                    'n_estimators': n_estimators,
                    'nthread':64,
                    'n_gpus':1,
                    'seed':42
                }

                estimator = XGBClassifier(
                    **xgb_est_params
                )


                uplift_model_cl_tr = ClassTransformation(
                    estimator=estimator
                )

                X_train_tmp = X_train.drop(columns=['group', 'response_att'])
                X_test_tmp = X_test.drop(columns=['group', 'response_att'])

                uplift_model_cl_tr.fit(
                    X=X_train_tmp,
                    y=X_train['response_att'],
                    treatment=X_train['group']
                )

                uplift_ts = uplift_model_cl_tr.predict(
                    X_test_tmp
                )

                df_submit = X_test_tmp.assign(uplift=uplift_ts)[['uplift']]

                print(f'Submit data shape: {df_submit.shape}\n')
                df_submit.head(2)

                df_submit['group'] = X_test['group']
                df_submit['response_att'] = X_test['response_att']
                score = custom_metric(df_submit)
                print('score ', score)
                if score > max_score:
                    max_score = score
                    run.tag('Good')
                run.log(name='score', value=score)

        
        #joblib.dump(value=uplift_model_cl_tr, filename='outputs/model' + str(i) + '.pkl')
        #i += 1

lr 0.17
depth 4


/anaconda/envs/azureml_py36_automl/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning:

It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.



Submit data shape: (206109, 1)

score  6.597657198173026
lr 0.17
depth 4


/anaconda/envs/azureml_py36_automl/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning:

It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.



Submit data shape: (206109, 1)

score  6.589259885699847
lr 0.17
depth 4


...

Submit data shape: (206109, 1)

score  6.5230085690392094
lr 0.17
depth 4


/anaconda/envs/azureml_py36_automl/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning:

It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.



Submit data shape: (206109, 1)

score  6.610550975874208
lr 0.17
depth 4


/anaconda/envs/azureml_py36_automl/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning:

It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.



Submit data shape: (206109, 1)

score  6.561083315127661


In [11]:
runs = {}
run_metrics = {}

# Create dictionaries containing the runs and the metrics for all runs containing the 'mse' metric
for r in tqdm(experiment.get_runs()):
    metrics = r.get_metrics()
    if 'score' in metrics.keys():
        runs[r.id] = r
        run_metrics[r.id] = metrics

# Find the run with the best (lowest) mean squared error and display the id and metrics
best_run_id = max(run_metrics, key = lambda k: run_metrics[k]['score'])
best_run = runs[best_run_id]
print('Best run is:', best_run_id)
print('Metrics:', run_metrics[best_run_id])

# Tag the best run for identification later
best_run.tag("Best Run")

22it [00:05,  4.05it/s]


Best run is: fb524d86-f5f6-4a3d-a3cb-4e8d66e11f6a
Metrics: {'lr': 0.2, 'score': 6.087121555088201}
